In [ ]:
!pip install transformers
import transformers

### Using pre-trained transformers
_for fun and profit_

There are many toolkits that let you access pre-trained transformer models, but the most powerful and convenient by far is [`huggingface/transformers`](https://github.com/huggingface/transformers). In this week's practice, you'll learn how to download, apply and modify pre-trained transformers for a range of tasks. Buckle up, we're going in!


__Pipelines:__ if all you want is to apply a pre-trained model, you can do that in one line of code using pipeline. Huggingface/transformers has a selection of pre-configured pipelines for masked language modelling, sentiment classification, question aswering, etc. ([see full list here](https://huggingface.co/transformers/main_classes/pipelines.html))

A typical pipeline includes:
* pre-processing, e.g. tokenization, subword segmentation
* a backbone model, e.g. bert finetuned for classification
* output post-processing

Let's see it in action:

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

print(classifier("BERT is amazing!"))


[{'label': 'POSITIVE', 'score': 0.9998860955238342}]


In [ ]:
import base64
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

# YOUR CODE: predict sentiment for each noble house and create outputs dict
outputs = {}
for house_name in data.keys():
  outputs[house_name] = classifier(data[house_name])[0]['label'] == 'POSITIVE'
#outputs = <YOUR CODE: dict (house name) : True if positive, False if negative>

assert sum(outputs.values()) == 3 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False
print("Well done!")

Well done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
  from ipykernel import kernelapp as app


You can also access vanilla Masked Language Model that was trained to predict masked words. Here's how:

In [ ]:
mlm_model = pipeline('fill-mask', model="bert-base-uncased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


P=0.99719 [CLS] donald trump is the president of the united states. [SEP]
P=0.00024 [CLS] donald duck is the president of the united states. [SEP]
P=0.00022 [CLS] donald ross is the president of the united states. [SEP]
P=0.00020 [CLS] donald johnson is the president of the united states. [SEP]
P=0.00018 [CLS] donald wilson is the president of the united states. [SEP]


In [ ]:
# Your turn: use bert to recall what year was the Soviet Union founded in
mlm_model(f"USSR was formed in {MASK} AD")

[{'score': 0.019533371552824974,
  'sequence': '[CLS] ussr was formed in 1992 ad [SEP]',
  'token': 2826,
  'token_str': '1992'},
 {'score': 0.017336530610919,
  'sequence': '[CLS] ussr was formed in 1946 ad [SEP]',
  'token': 3918,
  'token_str': '1946'},
 {'score': 0.016023412346839905,
  'sequence': '[CLS] ussr was formed in 1947 ad [SEP]',
  'token': 4006,
  'token_str': '1947'},
 {'score': 0.01600281149148941,
  'sequence': '[CLS] ussr was formed in 1991 ad [SEP]',
  'token': 2889,
  'token_str': '1991'},
 {'score': 0.014276874251663685,
  'sequence': '[CLS] ussr was formed in 1993 ad [SEP]',
  'token': 2857,
  'token_str': '1993'}]

```

```

```

```


Huggingface offers hundreds of pre-trained models that specialize on different tasks. You can quickly find the model you need using [this list](https://huggingface.co/models).


In [ ]:
text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""
ner_model = pipeline('ner')

# Task: create a pipeline for named entity recognition, use task name 'ner' and search for the right model in the list
#ner_model = <YOUR CODE>

named_entities = ner_model(text)

In [ ]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

OUTPUT: [{'word': 'Google', 'score': 0.8803111910820007, 'entity': 'I-MISC', 'index': 19, 'start': 73, 'end': 79}, {'word': 'Rose', 'score': 0.9005067944526672, 'entity': 'I-MISC', 'index': 27, 'start': 112, 'end': 116}, {'word': '##tta', 'score': 0.9509623646736145, 'entity': 'I-MISC', 'index': 28, 'start': 116, 'end': 119}, {'word': 'Guardian', 'score': 0.9992534518241882, 'entity': 'I-ORG', 'index': 40, 'start': 179, 'end': 187}, {'word': 'Ian', 'score': 0.9992009401321411, 'entity': 'I-PER', 'index': 46, 'start': 207, 'end': 210}, {'word': 'Sam', 'score': 0.999500036239624, 'entity': 'I-PER', 'index': 47, 'start': 211, 'end': 214}, {'word': '##ple', 'score': 0.9964978694915771, 'entity': 'I-PER', 'index': 48, 'start': 214, 'end': 217}, {'word': 'Stuart', 'score': 0.9991856217384338, 'entity': 'I-PER', 'index': 53, 'start': 240, 'end': 246}, {'word': 'Clark', 'score': 0.99964839220047, 'entity': 'I-PER', 'index': 54, 'start': 247, 'end': 252}, {'word': 'Germany', 'score': 0.99982106

### The building blocks of a pipeline

Huggingface also allows you to access its pipelines on a lower level. There are two main abstractions for you:
* `Tokenizer` - converts from strings to token ids and back
* `Model` - a pytorch `nn.Module` with pre-trained weights

You can use such models as part of your regular pytorch code: insert is as a layer in your model, apply it to a batch of data, backpropagate, optimize, etc.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [ ]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [ ]:
# You can now apply the model to get embeddings
with torch.no_grad():
    token_embeddings, sentence_embedding = model(**tokens_info)

print(sentence_embedding)

pooler_output


### The search for similar questions.

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to **really** solve this task using context-aware embeddings.

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2021-01-21 04:11:21--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2021-01-21 04:11:21--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdc53f4fb2c02b31450f5b6a113.dl.dropboxusercontent.com/cd/0/get/BHZ6nU6Nk1jy2TZIcKCGkjLbb8_iGp2puDOGEKoHTtbqXzXo7_oGRL_9em3l0y5K27ns6lvP4aKtldsaKK5aI4eNvJtdvhw9pZlBFJzU5sDlTQ/file?dl=1# [following]
--2021-01-21 04:11:22--  https://ucdc53f4fb2c02b31450f5b6a113.dl.dropboxusercontent.com/cd/0/get/BHZ6nU6Nk1jy2TZIcKCGkjLbb8_iGp2puDOGEKoHTtbqXzXo7_oGRL_9em3l0y5K27ns6lvP4aKtldsaKK5aI4eNvJtdvhw9pZlBFJzU5sDlTQ/file?dl=1
Resolving ucdc53f4

In [ ]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
model = GPT2LMHeadModel.from_pretrained('gpt2').train(False).to(device)

text = "The Fermi paradox "
tokens = tokenizer.encode(text)
num_steps = 1024
line_length, max_length = 0, 70

print(end=tokenizer.decode(tokens))

for i in range(num_steps):
    with torch.no_grad():
        logits = model(torch.as_tensor([tokens], device=device))[0]
    p_next = torch.softmax(logits[0, -1, :], dim=-1).data.cpu().numpy()

    next_token_index = p_next.argmax() #<YOUR CODE: REPLACE THIS LINE>
    # YOUR TASK: change the code so that it performs nucleus sampling

    tokens.append(int(next_token_index))
    print(end=tokenizer.decode(tokens[-1]))
    line_length += len(tokenizer.decode(tokens[-1]))
    if line_length >= max_length:
        line_length = 0
        print()




 The Fermi paradox  is a paradox that is not a paradox at all. It is a paradox that is not
 a paradox at all. It is a paradox that is not a paradox at all. It is
 a paradox that is not a paradox at all. It is a paradox that is not a
 paradox at all. It is a paradox that is not a paradox at all. It is a
 paradox that is not a paradox at all. It is a paradox that is not a paradox
 at all. It is a paradox that is not a paradox at all. It is a paradox
 that is not a paradox at all. It is a paradox that is not a paradox at
 all. It is a paradox that is not a paradox at all. It is a paradox that
 is not a paradox at all. It is a paradox that is not a paradox at all
. It is a paradox that is not a paradox at all. It is a paradox that is
 not a paradox at all. It is a paradox that is not a paradox at all. It
 is a paradox that is not a paradox at all. It is a paradox that is not
 a paradox at all. It is a paradox that is not a paradox at all. It is
 a paradox that is not a paradox at all. It 

KeyboardInterrupt: ignored

Transformers knowledge hub: https://huggingface.co/transformers/